In [1]:
from pennylane import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.insert(0, '/Users/tak/Github/QEmbedding/')
import Hybrid_nn
import torch
from torch import nn
import data
import pennylane as qml
import embedding

In [2]:
dev = qml.device('default.mixed', wires=4)

In [3]:

PATH_Model2 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model2 pre-train/Model2_48.pt'
PATH_Model3 = '/Users/tak/Github/QEmbedding/Results/QCNN_demonstration/Real device/Real/Model3 pre-train/Model3_48.pt'

In [4]:
class x_transform2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack2 = nn.Sequential(
            nn.Linear(4, 12),
            nn.ReLU(),
            nn.Linear(12, 12),
            nn.ReLU(),
            nn.Linear(12, 7)
        )
        
    def forward(self, x):
        x = self.linear_relu_stack2(x)
        return x.detach().numpy()


class x_transform3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Layer2: 14 * 14 -> 7 * 7
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 1, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Fully connected Layers 7 * 7 -> 7
        self.fc = torch.nn.Linear(7 * 7, 7, bias=True)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 7 * 7)
        x = self.fc(x)
        return x.detach().numpy()

model2 = x_transform2()
model3 = x_transform3()

In [38]:
@qml.qnode(dev)
def overlap_circuit1(x1,x2):
    embedding.Noisy_Four_QuantumEmbedding1(x1)
    embedding.Noisy_Four_QuantumEmbedding1_inverse(x2)
    return qml.probs(wires=range(4))

@qml.qnode(dev)
def overlap_circuit2(x1,x2):
    embedding.Noisy_Four_QuantumEmbedding2(x1)
    embedding.Noisy_Four_QuantumEmbedding2_inverse(x2)
    return qml.probs(wires=range(4))

def kernel(x1, x2, NQE):
    if NQE == False:
        kernel = overlap_circuit1(x1,x2)
    
    
    elif NQE == 'PCA-NQE':
        model2.load_state_dict(torch.load(PATH_Model2))
        x1 = model2(torch.tensor(x1))
        x2 = model2(torch.tensor(x2))
        kernel = overlap_circuit2(x1, x2)

    elif NQE == 'NQE':
        model3.load_state_dict(torch.load(PATH_Model3))
        x1 = model3(torch.tensor(x1))
        x2 = model3(torch.tensor(x2))
        kernel = overlap_circuit2(x1,x2)
    
    return kernel

  

In [72]:
feature_reduction = 'PCA4'
classes = [0,1]
X_train, X_test, Y_train, Y_test = data.data_load_and_process('mnist', feature_reduction=feature_reduction, classes=classes)
#X_train, X_test = torch.tensor(X_train).to(torch.float32), torch.tensor(X_test).to(torch.float32)
Y_train = [-1 if y == 0 else 1 for y in Y_train]
Y_test = [-1 if y == 0 else 1 for y in Y_test]

In [73]:
X_train = X_train[:200]
Y_train = Y_train[:200]

In [74]:
N = len(X_train)

In [75]:
Kernel_without_NQE = np.zeros((N,N))
Kernel_PCA_NQE = np.zeros((N,N))
Kernel_NQE = np.zeros((N,N))

In [76]:
for i in range(N):
    for j in range(i,N):
        x = kernel(X_train[i], X_train[j], False)
        Kernel_without_NQE[i][j] = x[0]
        Kernel_without_NQE[j][i] = x[0]

        x = kernel(X_train[i], X_train[j], "PCA-NQE")
        Kernel_PCA_NQE[i][j] = x[0]
        Kernel_PCA_NQE[j][i] = x[0]

In [63]:
Kernel0_MP = np.linalg.pinv(Kernel_without_NQE, hermitian = True)
Kernel2_MP = np.linalg.pinv(Kernel_PCA_NQE ,hermitian = True)

Y_train = np.array(Y_train)

print(np.sqrt(Y_train @ Kernel0_MP @ Y_train.T / N))
print(np.sqrt(Y_train @ Kernel2_MP @ Y_train.T / N))



276692.438709325
2534.0521965742473
